## Import All the Library

In [44]:
import os 
import io
import matplotlib.pyplot as plt
from google.cloud import vision
from google.cloud.vision import enums
import cv2 as cv

from PIL import Image
import urllib.request

import gzip

from google.cloud.vision_v1.proto import image_annotator_pb2
from google.protobuf.json_format import MessageToDict

import re
from google.cloud import vision
from google.cloud import storage
from google.protobuf import json_format
from google.cloud import storage

from google.cloud import language_v1
from google.cloud.language_v1 import enums

import argparse
import io
import json
import os

from google.cloud import language
import numpy
import six

## Set the Google application Credential

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="my-project-12195-46fed19566f2.json"

## 1-Classify a ‘text string’ passed by the user

**Step to follow**

**1-Classifying Content in a String**

**2-Create a function that will take text_content as an input**

**3-text_content the text content to analyze. Must include at least 20 words.**

**4-Use LanguageServiceClient() for text analysis operations such as sentiment analysis and entityrecognition.**

**5-Create a dictionary of document and passed the tex_content ,type as plain text and language as english**

**6-Use classifyText requests, which classifies content into categories along with a confidence score**

**7-Print the category name and confidence score**

In [34]:
def sample_classify_text(text_content):
    
    client = language_v1.LanguageServiceClient()

    # text_content = 'That actor on TV makes movies in Hollywood and also stars in a variety of popular new TV shows.'

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}

    response = client.classify_text(document)
    # Loop through classified categories returned from the API
    for category in response.categories:
        # Get the name of the category representing the document.
        # See the predefined taxonomy of categories:
        # https://cloud.google.com/natural-language/docs/categories
        print(u"Category name: {}".format(category.name))
        # Get the confidence. Number representing how certain the classifier
        # is that this category represents the provided text.
        print(u"Confidence: {}".format(category.confidence))


### Call the function and pass the Text string 

#### Below text give the description of Mobile phone

In [60]:
text_content = "A mobile phone is a portable device that can make and receive calls over a radio frequency link while the user is moving within a telephone service area. The radio frequency link establishes a connection to the switching systems of a mobile phone operator, which provides access to the public switched telephone network (PSTN)."
sample_classify_text(text_content)

Category name: /Internet & Telecom/Mobile & Wireless
Confidence: 0.6299999952316284
Category name: /Internet & Telecom/Service Providers
Confidence: 0.5199999809265137


**Result**

The Code correctly Predict the category as mobile and wireless  

## 2-Classify text  files in a local directory

#### Create a Function that will take text file as an input and predict category ,confidence score

In [36]:
def sample_classify_file(file_name): 
    file=open(file_name,'r')
    text_content=file.read()
    client = language_v1.LanguageServiceClient()
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}
    response = client.classify_text(document)
    for category in response.categories:
        print(u"Category name: {}".format(category.name))
        print(u"Confidence: {}".format(category.confidence))

#### Call the function and pass the text file

In [38]:
file_name='mobile.txt'
sample_classify_file(file_name)

Category name: /Internet & Telecom/Mobile & Wireless
Confidence: 0.6299999952316284
Category name: /Internet & Telecom/Service Providers
Confidence: 0.5199999809265137


## 3-Classify .pdf and .txt files stored on google Storage Bucket 

#### Create a function that will take url of the text/pdf stored in google storage and return  

In [9]:
def sample_classify_uri(gcs_content_uri):
    client = language_v1.LanguageServiceClient()
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"gcs_content_uri": gcs_content_uri, "type": type_, "language": language}
    response = client.classify_text(document)
    for category in response.categories:
        print(u"Category name: {}".format(category.name))
        print(u"Confidence: {}".format(category.confidence))

#### Call the function and pass the link of text file stored in google storage

In [10]:
gcs_content_uri = "gs://training-12195/textanalysis.txt"
sample_classify_uri(gcs_content_uri)

Category name: /Arts & Entertainment/Movies
Confidence: 0.9599999785423279


#### Call the function and pass the link of PDF file stored in google storage

In [28]:
gcs_content_uri="gs://training-12195/text.pdf"
sample_classify_uri(gcs_content_uri)

Category name: /Computers & Electronics/Software
Confidence: 0.7099999785423279


## 4-Index multiple text files

The index function in the tutorial script takes, as input, a directory containing multiple text files, and the path to a file where it stores the indexed output (the default file name is index.json). The index function reads the content of each text file in the input directory, and then passes the text files to the Cloud Natural Language API to be classified into content categories.

**Step**

**1-Create a function index that will take folder where all the text file are stored and index_file by where our json file will be stored**

**2-Iterate over each text file extract the content and call classify function to get the category and Confidence Score**

**3-Classify function will return Category and Confidence score and we will save for every text in json format**


In [46]:
def classify(text, verbose=True):
    """Classify the input text into categories. """

    language_client = language.LanguageServiceClient()

    document = language.types.Document(
        content=text,
        type=language.enums.Document.Type.PLAIN_TEXT)
    response = language_client.classify_text(document)
    categories = response.categories

    result = {}

    for category in categories:
        # Turn the categories into a dictionary of the form:
        # {category.name: category.confidence}, so that they can
        # be treated as a sparse vector.
        result[category.name] = category.confidence

    if verbose:
        print(text)
        for category in categories:
            print(u'=' * 20)
            print(u'{:<16}: {}'.format('category', category.name))
            print(u'{:<16}: {}'.format('confidence', category.confidence))

    return result

In [49]:
def index(path, index_file):
    """Classify each text file in a directory and write
    the results to the index_file.
    """

    result = {}
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)

        if not os.path.isfile(file_path):
            continue

        try:
            with io.open(file_path, 'r') as f:
                text = f.read()
                categories = classify(text, verbose=False)

                result[filename] = categories
        except Exception:
            print('Failed to process {}'.format(file_path))

    with io.open(index_file, 'w', encoding='utf-8') as f:
        f.write(json.dumps(result, ensure_ascii=False))

    print('Texts indexed in file: {}'.format(index_file))
    print()
    return result

In [50]:
path = 'Text_file'
index_file = 'index'
index(path, index_file)

Texts indexed in file: index



{'Android.txt': {'/Computers & Electronics': 0.800000011920929,
  '/Internet & Telecom/Mobile & Wireless/Mobile Apps & Add-Ons': 0.6499999761581421},
 'Cloud Computing.txt': {'/Computers & Electronics/Networking': 0.6800000071525574,
  '/Internet & Telecom/Web Services': 0.5400000214576721},
 'harry potter.txt': {"/Books & Literature/Children's Literature": 0.9200000166893005,
  "/People & Society/Kids & Teens/Children's Interests": 0.8500000238418579,
  '/People & Society/Subcultures & Niche Interests': 0.5600000023841858},
 'matildata.txt': {"/Books & Literature/Children's Literature": 0.9900000095367432,
  "/People & Society/Kids & Teens/Children's Interests": 0.9700000286102295},
 'Wireless.txt': {'/Internet & Telecom': 0.7099999785423279}}

## 5-Find files in a directory that are most similar to a query label passed by the user

Query with category labels
Once the index file (default file name = index.json) has been created, we can make queries to the index to retrieve some of the filenames and their confidence scores.

One way to do this is to use a category label as the query, which the tutorial accomplishes with the query_category function. The implementation of the helper functions, such as similarity, can be found in the classify_text_tutorial.py file. In your applications the similarity scoring and ranking should be carefully designed around specific use cases.

**STEP**

**1-Create a function query_category that will take Jason file we have created,category_string want to search as an input**

**2-Open the jason file and call the similarity function that will take the query category and the category prsent in json**

**3-Inside the similarity function we will call the split function that will calculate the similarities inside the subcategory**


    """The category labels are of the form "/a/b/c" up to three levels,
    for example "/Computers & Electronics/Software", and these labels
    are used as keys in the categories dictionary, whose values are
    confidence scores.
    The split_labels function splits the keys into individual levels
    while duplicating the confidence score, which allows a natural
    boost in how we calculate similarity when more levels are in common.
    Example:
    If we have
    x = {"/a/b/c": 0.5}
    y = {"/a/b": 0.5}
    z = {"/a": 0.5}
    Then x and y are considered more similar than y and z.
    """
    
**4-The split function will be called twice for both the category and than we will normalize the output for both**

**5- Finally will calculate cosine similarity and confidence score**

#### Create query_category Function

In [65]:
def query_category(index_file, category_string, n_top=3):
    """Find the indexed files that are the most similar to
    the query label.

    The list of all available labels:
    https://cloud.google.com/natural-language/docs/categories
    """

    with io.open(index_file, 'r') as f:
        index = json.load(f)

    # Make the category_string into a dictionary so that it is
    # of the same format as what we get by calling classify.
    query_categories = {category_string: 1.0}
    print(index)
    similarities = []
    for filename, categories in six.iteritems(index):
        similarities.append(
            (filename, similarity(query_categories, categories)))

    similarities = sorted(similarities, key=lambda p: p[1], reverse=True)

    print('=' * 20)
    print('Query: {}\n'.format(category_string))
    print('\nMost similar {} indexed texts:'.format(n_top))
    for filename, sim in similarities[:n_top]:
        print('\tFilename: {}'.format(filename))
        print('\tSimilarity: {}'.format(sim))
        print('\n')

    return similarities

#### Create similarities function

In [66]:

def similarity(categories1, categories2):
    """Cosine similarity of the categories treated as sparse vectors."""
    categories1 = split_labels(categories1)
    categories2 = split_labels(categories2)

    norm1 = numpy.linalg.norm(list(categories1.values()))
    norm2 = numpy.linalg.norm(list(categories2.values()))

    # Return the smallest possible similarity if either categories is empty.
    if norm1 == 0 or norm2 == 0:
        return 0.0

    # Compute the cosine similarity.
    dot = 0.0
    for label, confidence in six.iteritems(categories1):
        dot += confidence * categories2.get(label, 0.0)

    return dot / (norm1 * norm2)

#### Create split_labels function

In [67]:
def split_labels(categories):
    
    _categories = {}
    for name, confidence in six.iteritems(categories):
        labels = [label for label in name.split('/') if label]
        for label in labels:
            _categories[label] = confidence

    return _categories

#### Call query_category function pass the jason file and category string we want to query

In [68]:
index_file='index'
category_string = "/Internet & Telecom/Mobile & Wireless"
Similarities=query_category(index_file, category_string)
print(Similarities)

{'Android.txt': {'/Computers & Electronics': 0.800000011920929, '/Internet & Telecom/Mobile & Wireless/Mobile Apps & Add-Ons': 0.6499999761581421}, 'Cloud Computing.txt': {'/Computers & Electronics/Networking': 0.6800000071525574, '/Internet & Telecom/Web Services': 0.5400000214576721}, 'harry potter.txt': {"/Books & Literature/Children's Literature": 0.9200000166893005, "/People & Society/Kids & Teens/Children's Interests": 0.8500000238418579, '/People & Society/Subcultures & Niche Interests': 0.5600000023841858}, 'matildata.txt': {"/Books & Literature/Children's Literature": 0.9900000095367432, "/People & Society/Kids & Teens/Children's Interests": 0.9700000286102295}, 'Wireless.txt': {'/Internet & Telecom': 0.7099999785423279}}
Query: /Internet & Telecom/Mobile & Wireless


Most similar 3 indexed texts:
	Filename: Wireless.txt
	Similarity: 0.7071067811865476


	Filename: Android.txt
	Similarity: 0.6655735790452499


	Filename: Cloud Computing.txt
	Similarity: 0.31094107737232335


[

**Result**

We have five text file present in the folder out of which wirelesse.txt is having the highest similarity with the query category
followed by android.txt followed by cloud computing.txt.

The remaining two file is having no similarity with the query category
